1. As in your previous assignments, compare the accuracy of at least two
recommender system algorithms against your offline data.
2. Implement support for at least one business or user experience goal such as
increased serendipity, novelty, or diversity.
3. Compare and report on any change in accuracy before and after you’ve made the
change in #2.
4. As part of your textual conclusion, discuss one or more additional experiments that
could be performed and/or metrics that could be evaluated only if online evaluation
was possible. Also, briefly propose how you would design a reasonable online
evaluation environment. 

This project, uses data from Institut für Informatik and contains user ratings for various books: http://www2.informatik.uni-freiburg.de/~cziegler/BX/

The recommender systems are similar to ones used in previous assignments

In [2]:
!pip install surprise

In [0]:
#load libraries
import pandas as pd
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix 
import csv
import surprise
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate

#import data
reviews = []
openfile =open('BX-Book-Ratings.csv', 'r', newline='', encoding='ISO-8859-1')
r = csv.reader(openfile, delimiter = ';')
for i in r:
    reviews.append(i)
openfile.close()

reviews_df = pd.DataFrame(reviews)
header = reviews_df.iloc[0]
reviews_df = reviews_df.rename(columns = header)

#there are 36,000 books in the dataset. Going to only include the top 500 and bottom 200 rated books 
a = reviews_df['ISBN'].value_counts()
top_rated_books = a[0:500]
least_rated_books = a[36281:36281]
books = top_rated_books + least_rated_books
books = books.index 
reviews_df = reviews_df[reviews_df['ISBN'].isin(books)]

#there are 29,000 users in the dataset. Going to only include the top 5000 and bottom 4000 common users 
u = reviews_df['User-ID'].value_counts()
top_rated_users = u[0:5000]
least_rated_users = u[25202:29202]
users = top_rated_users + least_rated_users
users = users.index 
reviews_df = reviews_df[reviews_df['User-ID'].isin(users)]

In [0]:
#this dataset has a lot of null values, more than others I have been working with in this class. Therefore, it could be challening to obtain a good recommendation
user_matrix = (reviews_df.pivot_table(index=['User-ID'], 
                      columns=['ISBN'], 
                      values=['Book-Rating'],
                      aggfunc='first'))
user_matrix = user_matrix.astype(float)
user_matrix_na = user_matrix.copy()
user_matrix = user_matrix.fillna(1)

In [5]:
#Using SVD

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

R = user_matrix.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

U, s, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(s)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_matrix.columns, index = user_matrix.index)

preds_df.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


Book-Rating                        ...                                 
ISBN     002542730X 006016848X 0060173289  ... 1573225789 1573229326 1878424319
User-ID                                    ...                                 
100004     0.968942   1.043124   1.120346  ...   0.916396   0.991961   1.021588
100009     0.999877   1.147274   1.083304  ...   0.945633   0.739723   1.187031
10001      1.010174   0.997165   0.988170  ...   1.000370   1.015300   0.997773
100030     0.995360   0.996870   0.997332  ...   0.992878   0.997476   0.996337
100053     1.020840   1.019857   1.041798  ...   0.832059   1.007886   1.067784

[5 rows x 500 columns]

In [6]:
"""Calculate the RMSE"""
SE = (preds_df-user_matrix)*(preds_df-user_matrix)
MSE = SE.mean().mean()
RMSE = MSE ** (1/2)
"RMSE is " + str(RMSE)

'RMSE is 0.48518449738930386'

In [8]:
from surprise.model_selection import train_test_split
from surprise import accuracy

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(reviews_df[['User-ID', 'ISBN', 'Book-Rating']], reader)

# sample random trainset and testset
trainset, testset = train_test_split(data, test_size=.25)

# We'll use the famous SVD algorithm.
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 3.7607


3.7606617968896465

In [35]:
#See if better than Random

df_rand = 5 - pd.DataFrame(np.random.randn(user_rating_na.shape[0],user_rating_na.shape[1]), columns= user_rating_na.columns, index = user_rating_na.index )

"""Calculate the RMSE"""
SE = (df_rand-user_matrix)*(df_rand-user_matrix)
MSE = SE.mean().mean()
RMSE = MSE ** (1/2)
"RMSE is " + str(RMSE)

'RMSE is 4.131758822433176'

In [0]:
#Increased Serendipity By Adding Random Data Variables to Skew dataset

user_rating_na = user_matrix.mask(np.random.random(user_matrix.shape) < .1)
df_rand = 5 - pd.DataFrame(np.random.randn(user_rating_na.shape[0],user_rating_na.shape[1]), columns= user_rating_na.columns, index = user_rating_na.index )
user_rating_rand = user_rating_na.fillna(df_rand)

In [37]:
#Using SVD

from scipy.sparse.linalg import svds
from scipy.sparse import csr_matrix

R = user_rating_rand.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

U, s, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(s)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = user_matrix.columns, index = user_matrix.index)

preds_df.head(5)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


Book-Rating                        ...                                 
ISBN     002542730X 006016848X 0060173289  ... 1573225789 1573229326 1878424319
User-ID                                    ...                                 
100004     1.574741   1.835344   2.411033  ...   1.449370   1.775769   1.866581
100009     1.086284   1.148694   1.839725  ...   1.591033   1.622336   1.763124
10001      1.696376   1.094330   2.196722  ...   1.626798   1.536803   1.611266
100030     1.410620   1.007396   0.990442  ...   1.894345   1.258396   1.333742
100053     0.921277   1.801306   1.434233  ...   1.373822   1.554621   1.459173

[5 rows x 500 columns]

In [38]:
"""Calculate the RMSE"""
SE = (preds_df-user_matrix)*(preds_df-user_matrix)
MSE = SE.mean().mean()
RMSE = MSE ** (1/2)
"RMSE is " + str(RMSE)

'RMSE is 0.7852547716849505'

The RMSE has increased by adding in random datapoints, however, it is still a good model. Adding 10% random data could help with serendipity, getting users to try books that are sightly out of their comfort zone, but might still be something that they are interested in reading.

As part of your textual conclusion, discuss one or more additional experiments that could be performed and/or metrics that could be evaluated only if online evaluation was possible. Also, briefly propose how you would design a reasonable online evaluation environment.

- One test might be A/B testing, where users are divided into groups and given recommendations from a variety of the recommendation systems here. If one group tends to click on or purchase more recommended books than another, it would be a good test to see which recommender system works best.